In [1]:
pip install transformers datasets peft accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 18.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 16.8 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer
model_name = "bigscience/bloomz-560m"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    trust_remote_code=True
    )

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/222 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/14.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/85.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/715 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.12G [00:00<?, ?B/s]

In [3]:
def generate_text(model, tokenizer, prompt_text, max_tokens):
   prompt_text = tokenizer(prompt_text, return_tensors="pt")
   output = model.generate(
       input_ids = prompt_text["input_ids"],
       attention_mask = prompt_text["attention_mask"],
       max_length = max_tokens,
       repetition_penalty = 1.5,
       eos_token_id = tokenizer.eos_token_id
       )

   return tokenizer.batch_decode(output, skip_special_tokens=True)


In [4]:
initail_output = generate_text(model,tokenizer,"I want you to act as a logistician. ", 100)
print('initial model output: ',initail_output)

initial model output:  ['I want you to act as a logistician.  You will be able to:  Analyze the data']


In [5]:
from datasets import load_dataset

dataset_prompt = "fka/awesome-chatgpt-prompts"


data_prompt = load_dataset(dataset_prompt)
data_prompt = data_prompt.map(lambda x:tokenizer(x["prompt"]), batched=True)
train_prompts = data_prompt["train"].select(range(30))

README.md:   0%|          | 0.00/339 [00:00<?, ?B/s]

prompts.csv:   0%|          | 0.00/104k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/203 [00:00<?, ? examples/s]

Map:   0%|          | 0/203 [00:00<?, ? examples/s]

In [6]:
print(data_prompt['train'][:5])

{'act': ['An Ethereum Developer', 'SEO Prompt', 'Linux Terminal', 'English Translator and Improver', '`position` Interviewer'], 'prompt': ['Imagine you are an experienced Ethereum developer tasked with creating a smart contract for a blockchain messenger. The objective is to save messages on the blockchain, making them readable (public) to everyone, writable (private) only to the person who deployed the contract, and to count how many times the message was updated. Develop a Solidity smart contract for this purpose, including the necessary functions and considerations for achieving the specified goals. Please provide the code and any relevant explanations to ensure a clear understanding of the implementation.', "Using WebPilot, create an outline for an article that will be 2,000 words on the keyword 'Best SEO prompts' based on the top 10 results from Google. Include every relevant heading possible. Keep the keyword density of the headings high. For each section of the outline, include 

In [7]:
import pandas as pd

df= pd.DataFrame(data_prompt['train'][:])
df.head()

,act,prompt,input_ids,attention_mask
0,An Ethereum Developer,Imagine you are an experienced Ethereum develo...,"[186402, 1152, 1306, 660, 72560, 28857, 167625...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
1,SEO Prompt,"Using WebPilot, create an outline for an artic...","[39312, 15202, 51, 46712, 15, 7932, 660, 67606...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
2,Linux Terminal,I want you to act as a linux terminal. I will ...,"[44, 4026, 1152, 427, 1769, 661, 267, 104105, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
3,English Translator and Improver,"I want you to act as an English translator, sp...","[44, 4026, 1152, 427, 1769, 661, 660, 7165, 24...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
4,`position` Interviewer,I want you to act as an interviewer. I will be...,"[44, 4026, 1152, 427, 1769, 661, 660, 33322, 2...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."


In [8]:
from peft import get_peft_model, PromptTuningConfig, TaskType, PromptTuningInit
tuning_config = PromptTuningConfig(
    task_type=TaskType.CAUSAL_LM,
    prompt_tuning_init=PromptTuningInit.RANDOM,
    num_virtual_tokens=4,
    tokenizer_name_or_path=model_name
)

peft_model = get_peft_model(model, tuning_config)

In [9]:
from google.colab import drive
drive.mount('/content/drive')



Mounted at /content/drive


In [14]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    use_cpu=True,
    output_dir = "/content/drive/MyDrive/prompt_tuning",
    run_name='',
    auto_find_batch_size=True,
    learning_rate= 0.005,
    num_train_epochs=5
    )


In [ ]:
from transformers import Trainer, DataCollatorForLanguageModeling

trainer = Trainer(
    model=peft_model,
    args=training_args,
    train_dataset=train_prompts,
    data_collator=DataCollatorForLanguageModeling(tokenizer, mlm=False)
    )
trainer.train()

ValueError: Got unexpected arguments: {'num_items_in_batch': 769}